# ONS Charts

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [235]:
import os
from datetime import date, datetime, timedelta

import unittest

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import common_core
import ons_core

## NumPy Helper Functions

Useful functionality such as moving average or rolling sum

In [236]:
def shiftRegistrations(data):
    """Shift registration data left by half a period"""
    
    # Final value is invalid (so not included in the convolution result) and needs to be zero
    result = np.append(np.convolve(data, np.array([0.5, 0.5]), mode="valid"), 0)
    
    return result

In [237]:
class TestShiftRegistrations(unittest.TestCase):
    '''Class to test rollingSum function'''   

    def testShift(self):
        '''Test processing of a list shorter than the window size'''

        actual = shiftRegistrations(np.arange(6))
        expected = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 0])

        self.assertEqual((actual == expected).all(), True)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [238]:
def getEstimatedOccurrences(cache):
    """Calculate missing values in cache"""

    estimates = {}
    
    # Use the ONS occurrences for England and Wales
    masterArea = common_core.ENGLAND_WALES

    # Simple references to the registrations and occurrences ofthe master area
    knownRegistrations = shiftRegistrations(cache[masterArea][ons_core.TOTAL_REGISTRATIONS])
    knownOccurrences = cache[masterArea][ons_core.TOTAL_OCCURRENCES]

    # Run this process for all regions and nations other than the master area
    for areaName in common_core.regionNames + common_core.nationNames:
        if areaName != masterArea and areaName in cache:
            
            # TODO - remove COVID deaths before doing calculation

            # Shift registrations left by half a week
            shiftedRegistrations = shiftRegistrations(cache[areaName][ons_core.TOTAL_REGISTRATIONS])

            # Estimate occurrences using a simple percentage of the known occurrences
            estimatedOccurrences = knownOccurrences * np.divide(shiftedRegistrations, knownRegistrations,
                                      out=np.zeros_like(shiftedRegistrations), where=knownRegistrations != 0)
            
            # Locate the last week in the cache where total_occurrences is populated
            cacheIdx = np.where(cache[areaName][ons_core.TOTAL_OCCURRENCES] > 0)[0][-1]
            
            # Patch subsequent data using the estimates
            cache[areaName][cacheIdx + 1:][ons_core.TOTAL_OCCURRENCES] = estimatedOccurrences[cacheIdx + 1:]
            
            # Maintain a cache of estimates for testing purpopses
            estimates[areaName] = estimatedOccurrences
            
    return estimates

In [239]:
def calculateErrors(cache, estimates):
    """Calculate estimation errors"""

    masterArea = common_core.ENGLAND_WALES

    for areaName in common_core.regionNames + common_core.nationNames:
        if areaName in cache and areaName != masterArea:
            print(f"{areaName}:")
            
            totalPctMAE = 0
        
            years = range(2010, 2018)
            
            for year in years:
                startDate = f"{year}-01-01"
                stopDate = f"{year}-12-31"

                areaData = cache[areaName]

                startIdx = np.where(areaData[ons_core.WEEK_ENDED] >= startDate)[0][0]
                stopIdx = np.where(areaData[ons_core.WEEK_ENDED] < stopDate)[0][-1]

                pctMAE = 100 * np.average(np.abs(areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES] -
                                            areaData[startIdx:stopIdx][ons_core.TOTAL_REGISTRATIONS].astype(np.float64)) /
                                        areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES])

                pctMAE = 100 * np.average(np.abs(areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES] -
                                            shiftRegistrations(areaData[startIdx:stopIdx][ons_core.TOTAL_REGISTRATIONS])) /
                                        areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES])

                pctMAE = 100 * np.average(np.abs(areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES] -
                                            estimates[areaName][startIdx:stopIdx]) /
                                        areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES])

                totalPctMAE += pctMAE
                
                print(f"{year} = {pctMAE:.2f}%")

            print(f"Avg  = {totalPctMAE / len(years):.2f}%")
            print()

## Plot Data

Simple plots of ONS data

In [240]:
def prunePoints(x_points, y_points):
    '''Remove leading and trailing zeros'''

    match = np.where(y_points > 0)[0]

    if len(match) > 0:
        start = match[0]
        end = match[-1]
    else:
        start = 0
        end = -1
        
    return x_points[start:end + 1], y_points[start:end + 1]
    

def plotRegion(cache, minimums, maximums, averages, areaName, fields, fig, ax, startDate=None, stopDate=None):
    '''Plot data for visual inspection'''
    
    areaData = cache[areaName]

    ax.set_title(f"Weekly Deaths in {areaName}\nShown by date of occurrence")
    ax.set_ylabel('Number of deaths')

    # Determine which rows need to be plotted
    startIdx = np.where(areaData[ons_core.WEEK_ENDED] >= startDate)[0][0]
    stopIdx = np.where(areaData[ons_core.WEEK_ENDED] < stopDate)[0][-1]

    xMin = yMin = stopIdx
    xMax = yMax = 0
    
    for field in fields:
        y_points = areaData[field["name"]][startIdx:stopIdx + 1]
        x_points = np.arange(len(y_points))       
        x_points, y_points = prunePoints(x_points, y_points)
        ax.plot(x_points, y_points, label = field["label"], color=field["color"], linestyle=field["linestyle"])
        if len(x_points) > 0:
            xMin = min(xMin, min(x_points))
            yMin = min(yMin, min(y_points))
            xMax = max(xMax, max(x_points))
            yMax = max(yMax, max(y_points))

    y_points = averages[areaName][startIdx:stopIdx + 1]
    x_points = np.arange(len(y_points))       
    x_points, y_points = prunePoints(x_points, y_points)
    ax.plot(x_points, y_points, label = "averages", color="navy", linestyle="dotted")
    if len(x_points) > 0:
        xMin = min(xMin, min(x_points))
        yMin = min(yMin, min(y_points))
        xMax = max(xMax, max(x_points))
        yMax = max(yMax, max(y_points))

    y1_points = minimums[areaName][startIdx:stopIdx + 1]
    y2_points = maximums[areaName][startIdx:stopIdx + 1]
    x_points = np.arange(len(y1_points))       
    x_points, y1_points = prunePoints(x_points, y1_points)
    x_points = np.arange(len(y2_points))       
    x_points, y2_points = prunePoints(x_points, y2_points)
    ax.fill_between(x_points, y1_points, y2_points, label = "range", color="lightsteelblue")
    if len(x_points) > 0:
        xMin = min(xMin, min(x_points))
        yMin = min(yMin, min(y_points))
        xMax = max(xMax, max(x_points))
        yMax = max(yMax, max(y_points))

    # Determine y-axis labelling
    yMax *= 1.1
    if yMax > 10000:
        interval = yMax // 10 // 1000 * 1000
    elif yMax > 1000:
        interval = yMax // 10 // 100 * 100
    else:
        interval = yMax // 10

    ax.set_ylim(ymin=0, ymax=yMax)
    ax.set_yticks(np.arange(0, yMax, interval))
    ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Determine x-axis labelling
    xticks = np.array(areaData[ons_core.WEEK_ENDED])[startIdx:stopIdx + 1]
    tickInterval = (stopIdx - startIdx) // 52
    if tickInterval == 0:
        tickInterval = 1

    ax.set_xlim(xmin=xMin - (xMin - ax.get_xlim()[0]) / 5, xmax=xMax + (ax.get_xlim()[1] - xMax) / 5)
    ax.set_xticks(np.arange(0, len(xticks), tickInterval))
    ax.set_xticklabels(xticks[::tickInterval], rotation=90)

    lastDate = datetime.strptime(areaData[ons_core.WEEK_ENDED][stopIdx], "%Y-%m-%d").strftime("%A %-d %B %Y")

    textStr = f'n.b. This only shows deaths occurring before {lastDate}'
    ax.text(0.5, 0.88, textStr, transform=ax.transAxes, horizontalalignment='center', verticalalignment='top')

    #textStr = 'Sources: Office for National Statistics\nPublic Health England'
    textStr = 'Source: Office for National Statistics'
    ax.text(0.98, 0.04, textStr, transform=ax.transAxes, horizontalalignment='right', verticalalignment='bottom')

    textStr = f'Created {datetime.now().strftime("%d %b")}\n@Mike_aka_Logiqx'
    ax.text(0.02, 0.04, textStr, transform=ax.transAxes, horizontalalignment='left', verticalalignment='bottom')

    ax.legend(loc = 'upper center', ncol=3)


def plotRegions(cache, minimums, maximums, averages, areaNames, startDate=None, stopDate=None):
    '''Plot data for visual inspection'''
    
    fig, axs = plt.subplots(len(areaNames), figsize=(16, 6 * len(areaNames)), dpi=150)
    
    fields = [
        {
            "name": ons_core.TOTAL_OCCURRENCES,
            "label": "Total Occurrences",
            "color": "navy",
            "linestyle": "solid"
        }
    ]
    
    disabled = [
        {
            "name": ons_core.TOTAL_REGISTRATIONS,
            "label": "Total Registrations",
            "color": "cornflowerblue",
            "linestyle": "dotted"
        },
        {
            "name": ons_core.COVID_OCCURRENCES,
            "label": "COVID-19 Occurrences",
            "color": "red",
            "linestyle": "solid"
        },
        {
            "name": ons_core.COVID_REGISTRATIONS,
            "label": "COVID-19 Registrations",
            "color": "lightcoral",
            "linestyle": "dotted"
        }
    ]

    for i in range(len(areaNames)):
        areaName = areaNames[i]
        areaData = cache[areaName]

        if len(areaNames) == 1:
            plotRegion(cache, minimums, maximums, averages, areaName, fields, fig, axs, startDate=startDate, stopDate=stopDate)
        else:
            plotRegion(cache, minimums, maximums, averages, areaName, fields, fig, axs[i], startDate=startDate, stopDate=stopDate)

    plt.subplots_adjust(hspace=0.5)
    plt.show(fig)

## Interactive Testing

In [241]:
if __name__ == '__main__':

    cache = ons_core.loadCsvFiles(ons_core.ONS_DEATHS, "weekly", verbose = False)
    
    estimates = getEstimatedOccurrences(cache)

    #calculateErrors(cache, estimates)

In [242]:
def calculateAverages(cache):
    """Calculate 5 year averages and ranges"""

    minimums = {}
    maximums = {}
    averages = {}
    
    for areaName in list(cache.keys()):

        tmpMin = np.array([], dtype="u4")
        tmpMax = np.array([], dtype="u4")
        tmpAvg = np.array([])
        
        yearIdx = np.where(cache[areaName][ons_core.WEEK_NUMBER] == 1)[0]

        for i in range(5, len(yearIdx)):

            if i == len(yearIdx) - 1:
                numWeeks = len(cache[areaName]) - yearIdx[i]
            else:
                numWeeks = yearIdx[i + 1] - yearIdx[i]

            grid = np.vstack([
                cache[areaName][yearIdx[i - 5]:yearIdx[i - 5] + numWeeks][ons_core.TOTAL_OCCURRENCES],
                cache[areaName][yearIdx[i - 4]:yearIdx[i - 4] + numWeeks][ons_core.TOTAL_OCCURRENCES],
                cache[areaName][yearIdx[i - 3]:yearIdx[i - 3] + numWeeks][ons_core.TOTAL_OCCURRENCES],
                cache[areaName][yearIdx[i - 2]:yearIdx[i - 2] + numWeeks][ons_core.TOTAL_OCCURRENCES],
                cache[areaName][yearIdx[i - 1]:yearIdx[i - 1] + numWeeks][ons_core.TOTAL_OCCURRENCES]
            ])

            tmpMin = np.append(tmpMin, np.min(grid, axis = 0))
            tmpMax = np.append(tmpMax, np.max(grid, axis = 0))
            tmpAvg = np.append(tmpAvg, np.average(grid, axis = 0))
        
        minimums[areaName] = np.concatenate((np.zeros(yearIdx[5], dtype="u4"), tmpMin))
        maximums[areaName] = np.concatenate((np.zeros(yearIdx[5], dtype="u4"), tmpMax))
        averages[areaName] = np.concatenate((np.zeros(yearIdx[5]), tmpAvg))
        
    return minimums, maximums, averages


if __name__ == '__main__':
    
    minimums, maximums, averages = calculateAverages(cache)

In [ ]:
if __name__ == '__main__':

    areaNames = ["England and Wales", "England", "Wales"]
    areaNames = ["West Midlands", "East Midlands", "South West"]
    areaNames = ["North West", "North East", "Yorkshire and The Humber"]
    areaNames = ["London", "East of England", "South East"]

    # Check heatwave of August 2003 - https://www.eurosurveillance.org/content/10.2807/esm.10.07.00558-en
    startDate = '2003-07-01'
    stopDate = '2003-09-01'

    # Show reg delays
    startDate = date(2010, 1, 1).strftime('%Y-%m-%d')
    stopDate = date(2020, 1, 8)

    # Last X years
    startDate = (date(2021, 1, 15) - timedelta(weeks = 52 * 10)).strftime('%Y-%m-%d')
    stopDate = date(2021, 1, 15).strftime('%Y-%m-%d')

    # Same range as error check
    #startDate = '2014-01-01'
    #stopDate = '2018-12-31'
    
    plotRegions(cache, minimums, maximums, averages, areaNames, startDate=startDate, stopDate=stopDate)